# SCRAPING SET UP

In [3]:
# Import necessary modules
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from datetime import datetime
import pandas as pd
import inspect
import random
import time
import re

# FUNCTIONS

# Sleep
def fSleep(x, y):
    # x must be larger than y
    time.sleep(random.uniform(x, y))

# 5etools Start
def startScraping(url, type):
    # VARIABLES

    # Buttons
    button_filter = '//*[@id="filter-search-group"]/button[1]'
    button_all_sources = '/html/body/div[7]/div/div[2]/div[1]/div[1]/div[2]/div[2]/div[2]/button[1]'
    button_save_filters = '/html/body/div[7]/div/div[1]/div[2]/div[2]/button[3]'

    # Lists
    list_parent = '//*[@id="listcontainer"]/div[4]'

    # SCRAPING

    # Open the webpage

    # Instantiate an Options object
    option = webdriver.ChromeOptions()
    #Remove navigator.webdriver flag
    option.add_argument('--disable-blink-features=AutomationControlled')
    # Change the resolution of the browser
    option.add_argument("window-size=1920,1080")
    # Adjusting the user agent
    option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
    #Open Browser
    driver = webdriver.Chrome(executable_path='chromedriver.exe', options=option)
    # Open the specfied URL
    driver.get(url)
    # Sleep to avoid errors
    time.sleep(random.uniform(10.0, 15.0))

    # Set up the filters

    # Open the filter menu
    driver.find_element_by_xpath(button_filter).click()
    # Sleep to avoid errors
    fSleep(3, 5)
    # Set the filter to all sources
    driver.find_element_by_xpath(button_all_sources).click()
    # Sleep to avoid errors
    fSleep(3, 5)
    # Extra filter settings for items
    if type == 'item':
        
    # Save the filter settings
    driver.find_element_by_xpath(button_save_filters).click()
    # Sleep to avoid errors
    fSleep(3, 5)
    # Print noting success
    print('Filters set to all sources')
    # Return the driver to work with
    return driver, list_parent

IndentationError: expected an indented block (<ipython-input-3-a07cb8c1da75>, line 65)


# ITEMS

## Scraping

In [25]:
# Set up a DataFrame

# Open 5etools.com and set up the filters for the bestiary
driver, list_parent = startScraping('https://5e.tools/items.html#battleaxe%20armblade_erlw')

# BASIC VARIABLES

# Mundane table
tableMundane = '//*[@id="listcontainer"]/div[1]/div[4]'
# Magic Table
tableMagic = '//*[@id="listcontainer"]/div[2]/div[3]'

# Count the number of rows in Mundane
mundaneList = driver.find_element_by_xpath(tableMundane)
# 367 Mundane Items
vMundaneCount = len(mundaneList.find_elements_by_xpath("./*"))
# Print
print(f'Mundane Items: {vMundaneCount}')

# Count the number of rows in Magic
magicList = driver.find_element_by_xpath(tableMagic)
#1014 Magic Items
vMagicCount = len(magicList.find_elements_by_xpath("./*"))
# Print
print(f'Magic Items: {vMagicCount}')

# TABLE VARIABLES

# ROW 2
row2 = driver.find_element_by_xpath('//*[@id="pagecontent"]/tr[2]').text
workingName = row2.split("\n")
# Name
finalName = workingName[0]
# Source
finalSource = workingName[1]
# Page Number
finalPageNumber = workingName[2]

# ROW 3
row3 = driver.find_element_by_xpath('//*[@id="pagecontent"]/tr[3]').text
row3 = row3.split(" (")
rarity = row3[0].split(", ")
# Attunement Requirements
try:
    finalAttunement = row3[1].replace(")", "")
except:
    pass
# Rarity Tier
finalTier = rarity[0]
# Rarity
finalRarity = rarity[1]




# ----------------
# CLOSE THE DRIVER
# ----------------
driver.close()

Filters set to all sources
Mundane Items: 367
Magic Items: 1014


'p276'

## Data Cleaning
### For cleaning the list that I copy-paste from 5etools

### Magic Items

In [ ]:
# Import Pandas
import pandas as pd
# Read the CSV to a DataFrame
df = pd.read_csv('RAW DATA/Item Lists by Rarity - Magical Items.csv', header = None)
# Change the column name
df.columns = ['stuff']
# Create a new DataFrame for the final database
dbMagicItems = pd.DataFrame(columns= ['NAME', 'TYPE', 'WEIGHT', 'ATTUNEMENT', 'RARITY', 'SOURCE', 'LINK'])
# Set a margin to iterate through the index
margin = 0
# Loop through Database's rows and write to them from the CSV
while margin < len(df):
    vName = df['stuff'][0 + margin]
    vType = df['stuff'][1 + margin]
    vWeight = df['stuff'][2 + margin]    
    vAttunement = df['stuff'][3 + margin]
    if vAttunement != '×':
        vAttunement = 'N'
        vRarity = df['stuff'][3 + margin]
        vSource = df['stuff'][4 + margin]
        margin += 5
    else:
        vAttunement = 'Y'
        vRarity = df['stuff'][4 + margin]
        vSource = df['stuff'][5 + margin]
        margin += 6
    # Format the name for the link    
    vNameLink = vName.lower().strip().replace(" ", "%20")
    vNameLink = vNameLink.replace(",", "%2c")
    vNameLink = vNameLink.replace("+", "%2b")
    vNameLink = vNameLink + "_" + vSource.lower()
    # Build the link
    vLink = f'https://5e.tools/items.html#{vNameLink}'
    dbMagicItems = dbMagicItems.append({'NAME': vName, 'TYPE': vType, 'WEIGHT': vWeight, 'ATTUNEMENT': vAttunement, 'RARITY': vRarity, 'SOURCE': vSource, 'LINK': vLink}, ignore_index = True)
# Export to CSV
dbMagicItems.to_csv('DATA OUTPUT/magic_items.csv')

### Mundane (None) Items

In [ ]:
# Read the CSV into a DataFrame
dfMundane = pd.read_csv('RAW DATA/Item Lists by Rarity - Mundane Items.csv', header = None)
# Set the colum name
dfMundane.columns = ['stuff']
# Create a new DataFrame for the final database
dbMundane = pd.DataFrame(columns= ['NAME', 'TYPE', 'COST', 'WEIGHT', 'SOURCE', 'LINK'])
# Set a margin to iterate through the index
margin = 0
# Add rows to the DB while the margin is less than the DF's length
while margin < len(dfMundane):
    vName = dfMundane['stuff'][0 + margin]
    vType = dfMundane['stuff'][1 + margin]
    vCost = dfMundane['stuff'][2 + margin]
    vWeight = dfMundane['stuff'][3 + margin]
    vSource = dfMundane['stuff'][4 + margin]
    # Format the name for the link    
    vNameLink = vName.lower().strip().replace(" ", "%20")
    vNameLink = vNameLink.replace(",", "%2c")
    vNameLink = vNameLink.replace("+", "%2b")
    vNameLink = vNameLink + "_" + vSource.lower()
    # Build the link
    vLink = f'https://5e.tools/items.html#{vNameLink}'
    # Append a row to the Database
    dbMundane = dbMundane.append({'NAME': vName, 'TYPE': vType, 'COST': vCost, 'WEIGHT': vWeight, 'SOURCE': vSource, 'LINK': vLink}, ignore_index = True)
    # Increment the margin
    margin += 5
# Export to CSV
dbMundane.to_csv('DATA OUTPUT/mundane_items.csv')

# BESTIARY

## Scraping

In [80]:
# Set up a DataFrame

# Open 5etools.com and set up the filters for the bestiary
driver, list_parent = startScraping('https://5e.tools/bestiary.html#aarakocra_mm')

# VARIABLES

# Count the number of rows
myList = driver.find_element_by_xpath(list_parent)
# 1630 Bestiary entries
vRowCount = len(myList.find_elements_by_xpath("./*"))
# Print
print(vRowCount)

driver.close()

Filters set to all sources


## Data Cleaning
### For cleaning the list that I copy-paste from 5etools

In [ ]:
# TODO: Need to handle parentheticals, the subtype
    # E.G. Aberration (shapechanger)
        # Move the parenthteical to another column named SUBTYPE

# Import necessary modules
import pandas as pd
import re
# Read the CSV to a DataFrame
df = pd.read_csv('RAW DATA/Bestiary - Sheet1.csv', header = None)
# Change the column name
df.columns = ['stuff']
# Create a new DataFrame for the final database
dbBestiary = pd.DataFrame(columns= ['NAME', 'TYPE', 'SUBTYPE', 'CR', 'SOURCE', 'LINK'])
# Set a margin to iterate through the index
margin = 0
# Loop through Database's rows and write to them from the CSV
while margin < len(df):
    # Get the name
    vName = df['stuff'][0 + margin]
    # Get the Type
    vType = df['stuff'][1 + margin]
    # Parse the Type into Type and Subtype
    if '(' in vType:
        x = re.search('\s\(.*\)', vType)
        y = x.group(0).strip()
        vSubtype = re.sub('\(*\)*', '', y)
        vType = re.sub('\s\(.*\)', '', vType)
    else:
        vType = vType
        vSubtype = '' 
    # Get the CR
    vCR = df['stuff'][2 + margin]
    # Get the source
    vSource = df['stuff'][3 + margin]
    # Format the name for the link    
    vNameLink = vName.lower().strip().replace(" ", "%20")
    vNameLink = vNameLink.replace(",", "%2c")
    vNameLink = vNameLink.replace("+", "%2b")
    vNameLink = vNameLink + "_" + vSource.lower()
    # Build the link
    vLink = f'https://5e.tools/bestiary.html#{vNameLink}'
    dbBestiary = dbBestiary.append({'NAME': vName, 'TYPE': vType, 'SUBTYPE': vSubtype, 'CR': vCR, 'SOURCE': vSource, 'LINK': vLink}, ignore_index = True)
    # Increment the offset
    margin += 4
# Export to CSV
dbBestiary.to_csv('DATA OUTPUT/bestiary.csv', index = False)

# SPELLS

## Scraping (DONE)

In [ ]:
# Set up a DataFrame
dfSpells = pd.DataFrame(columns = ['NAME', 'SOURCE', 'PAGE_NUMBER', 'LEVEL', 'SCHOOL', 'CASTING_TIME', 'RANGE', 'COMPONENTS', 'COMPONENTS_MATERIALS', 'COMPONENETS_COST', 'DURATION', 'CLASSES', 'SUBCLASSES'])

# Open 5etools.com and set up the filters
driver, list_parent = startScraping("https://5e.tools/spells.html#abi-dalzim's%20horrid%20wilting_xge")

# VARIABLES

# Count the number of rows
myList = driver.find_element_by_xpath(list_parent)
vRowCount = len(myList.find_elements_by_xpath("./*"))
# Name
xName = '//*[@id="pagecontent"]/tr[2]'
# Level & School
xLevelSchool = '//*[@id="pagecontent"]/tr[3]'
# Casting Time
xCastingTime = '//*[@id="pagecontent"]/tr[4]'
# Range
xRange = '//*[@id="pagecontent"]/tr[5]'
# Components
xComponents = '//*[@id="pagecontent"]/tr[6]'
# Duration
xDuration = '//*[@id="pagecontent"]/tr[7]'
# Description
xDescription = '//*[@id="pagecontent"]/tr[9]'
# Classes
xClasses = '//*[@id="pagecontent"]/tr[10]/td/div[1]'
# Subclasses
xSubClasses = '//*[@id="pagecontent"]/tr[10]/td/div[2]'

# Loop through all the rows and scrape their data
for row in range(1, vRowCount + 1):
    print(f'Scraping item {row} of {vRowCount}')
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # SCRAPING
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Set up the row based on its index
    xRow = f'//*[@id="listcontainer"]/div[4]/div[{row}]'
    # Click into the row
    driver.find_element_by_xpath(xRow).click()
    # Sleep
    fSleep(0.5, 1)

    # ROW 2

    tName = driver.find_element_by_xpath(xName).text
    workingName = tName.split("\n")
    # Name
    finalName = workingName[0]
    # Source
    finalSource = workingName[1]
    # Page Number
    finalPageNumber = workingName[2]

    # ROW 3

    tLevelSchool = driver.find_element_by_xpath(xLevelSchool).text
    workingLevelSchool = tLevelSchool.split(" ")
    # Level 
    finalLevel = workingLevelSchool[0]
    # School
    finalSchool = workingLevelSchool[1]

    # ROW 4

    tCastingTime = driver.find_element_by_xpath(xCastingTime).text
    workingCastingTime = tCastingTime.split(": ")
    # Casting Time
    finalCastingTime = workingCastingTime[1]

    # ROW 5

    tRange = driver.find_element_by_xpath(xRange).text
    workingRange = tRange.split(": ")
    # Range
    finalRange = workingRange[1]

    # Components & Cost
    tComponents = driver.find_element_by_xpath(xComponents).text
    tComponents = tComponents.replace(")", "")
    workingComponents = tComponents.split(": ")
    workingComponents = workingComponents[1].split(" (")
    # Components Required
    finalComponentsRequired = workingComponents[0]
    # Material Components
    try:
        finalComponentsMaterial = workingComponents[1]
    except IndexError:
        finalComponentsMaterial = None

    # Description
    tDescription = driver.find_element_by_xpath(xDescription).text

    # Classes
    try:
        tClasses = driver.find_element_by_xpath(xClasses).text
        finalClasses = tClasses.split(": ")[1]
    except:
        finalClasses = None

    # Subclasses
    try:
        tSubClasses = driver.find_element_by_xpath(xSubClasses).text
        finalSubClasses = tSubClasses.split(": ")[1]
    except:
        finalSubClasses = None

    # Currency and Cost
    try:
        # Set up emtpy lists
        lCosts = []
        lCurrencies = []
        # Set up the material description for regex
        componentsMaterialsReg = finalComponentsMaterial.replace(",", "")
        # If "#### gp" exists in the string
        if len(re.findall("(\d+\s[cesgp]p)", componentsMaterialsReg)) >= 1:
            # Execute the regex
            lMatches = re.findall("(\d+\s[cesgp]p)", componentsMaterialsReg)
            # Set the string
            finalCost = ''
            # Loop through the matches
            for item in lMatches:    
                
                # DIGITS & CURRENCY

                # Pull out the cost
                cost = re.search("\d+", item)[0]
                # Append the cost to the cost list
                lCosts.append(cost)
                # Pull out the currency
                currency = item[-2:]
                # Append the currency to the currency list
                lCurrencies.append(currency)

            # Set the matches list to the components cost
            finalCost = ', '.join(lMatches)
        else:
            finalCost = None
    except:
        finalCost = None

    # ROW 7

    tDuration = driver.find_element_by_xpath(xDuration).text
    workingDuration = tDuration.split(": ")
    # Duration
    finalDuration = workingDuration[1]

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # DATAFRAME
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Append the row to the DF
    dfSpells = dfSpells.append({'NAME':finalName, 'SOURCE':finalSource, 'PAGE_NUMBER':finalPageNumber, 'LEVEL':finalLevel, 'SCHOOL':finalSchool, 'CASTING_TIME':finalCastingTime, 'RANGE':finalRange, 'COMPONENTS':finalComponentsRequired, 'COMPONENTS_MATERIALS':finalComponentsMaterial, 'COMPONENETS_COST':finalCost, 'DURATION':finalDuration, 'CLASSES': finalClasses, 'SUBCLASSES': finalSubClasses}, ignore_index = True)

    # Export to CSV
    dfSpells.to_csv('SPELLS.csv')

driver.close()

## Data Cleaning
### For cleaning the list that I copy-paste from 5etools

In [ ]:
# Import Pandas
import pandas as pd
# Read the CSV to a DataFrame
dfSpells = pd.read_csv('RAW DATA/Spells - Sheet1.csv', header = None)
# Change the column name
dfSpells.columns = ['stuff']
# Create a new DataFrame for the final database
dbSpells = pd.DataFrame(columns= ['NAME', 'LEVEL', 'RITUAL', 'TECHNOLOGY', 'CASTING_TIME', 'SCHOOL', 'CONCENTRATION', 'RANGE', 'SOURCE', 'LINK'])
# Set a margin to iterate through the index
margin = 0
# Loop through Database's rows and write to them from the CSV
while margin < len(dfSpells):
    vName = dfSpells['stuff'][0 + margin]
    vLevel = dfSpells['stuff'][1 + margin]
    # Parse the level
    if '(rit.)' in vLevel:
        x = re.search('\s\(.*\)', vLevel)
        y = x.group(0).strip()
        vRitual = 'Y'
        vLevel = re.sub('\s\(.*\)', '', vLevel)
        vTech = 'N'
    elif '(tec.)' in vLevel:
        x = re.search('\s\(.*\)', vLevel)
        y = x.group(0).strip()
        vTech = 'Y'
        vLevel = re.sub('\s\(.*\)', '', vLevel)
        vRitual = 'N'
    else:
        vLevel = vLevel
        vRitual = 'N'
        vTech = 'N'
    vCastingTime = dfSpells['stuff'][2 + margin]    
    vSchool = dfSpells['stuff'][3 + margin]
    vConcentration = dfSpells['stuff'][4 + margin]
    if vConcentration != '×':
        vConcentration = 'N'
        vRange = dfSpells['stuff'][4 + margin]
        vSource = dfSpells['stuff'][5 + margin]
        margin += 6
    else:
        vConcentration = 'Y'
        vRange = dfSpells['stuff'][5 + margin]
        vSource = dfSpells['stuff'][6 + margin]
        margin += 7
    # Format the name for the link    
    vNameLink = vName.lower().strip().replace(" ", "%20")
    vNameLink = vNameLink.replace(",", "%2c")
    vNameLink = vNameLink.replace("+", "%2b")
    vNameLink = vNameLink + "_" + vSource.lower()
    # Build the link
    vLink = f'https://5e.tools/spells.html#{vNameLink}'
    dbSpells = dbSpells.append({'NAME': vName, 'LEVEL': vLevel, 'RITUAL': vRitual, 'TECHNOLOGY': vTech,'CASTING_TIME': vCastingTime, 'SCHOOL': vSchool, 'CONCENTRATION': vConcentration, 'RANGE': vRange, 'SOURCE': vSource, 'LINK': vLink}, ignore_index = True)
# Export to CSV
dbSpells.to_csv('DATA OUTPUT/spells.csv', index = False)